In [1]:
import numpy as np
import netCDF4 as nc

In [2]:
data = nc.Dataset('../data/kaggle_solar/train/dswrf_sfc_latlon_subset_19940101_20071231.nc')
data

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format UNDEFINED):
    Conventions: CF-1.0
    title: Subset of data from 2nd-generation multi-decadal ensemble reforecast generated from the NCEP Global Ensemble Forecast System, mimicking version operational at NCEP/EMC circa mid-2012.
    institution: NOAA Earth System Research Laboratory (ESRL)
    source: NCEP GFS v 9.01, T254L42.  Control initial conditions from CFSRR.  Perturbed initial conditions from ETR.  Model error simulated with STTP.
    references: http://www.esrl.noaa.gov/psd/forecasts/reforecast2/index.html
    history: Subset created 2013-01-15 19:19:01 UTC
    comment: Original dataset generated on DOE's supercomputers at Lawrence Berkeley Laboratory through ALCC/ASCR grant.
    dimensions(sizes): time(5113), lat(9), lon(16), ens(11), fhour(5)
    variables(dimensions): float64 time(time), int32 intTime(time), float32 lat(lat), float32 lon(lon), int16 ens(ens), int32 fhour(fhour), int32 intValidTi

In [3]:
for v in data.variables.keys():
    print v

time
intTime
lat
lon
ens
fhour
intValidTime
Downward_Short-Wave_Rad_Flux


In [4]:
dswrf = data.variables.values()[-1]

In [5]:
dswrf

<type 'netCDF4._netCDF4.Variable'>
float32 Downward_Short-Wave_Rad_Flux(time, ens, fhour, lat, lon)
    _FillValue: 9999.0
    units: W m-2
    long_name: Downward_Short-Wave_Rad_Flux_Average (Average for  Mixed Intervals) @ surface
    cell_methods: time: mean
    GRIB_param_discipline: Meteorological_products
    GRIB_param_category: Short-wave_Radiation
    GRIB_param_name: Downward_short_wave_rad_flux
    GRIB_generating_process_type: Forecast
    GRIB_param_id: [  2   0   4 192]
    GRIB_product_definition_template: 8
    GRIB_product_definition_template_desc: Average, accumulation, extreme values or other statistically processed value at a horizontal level in a time interval
    GRIB_level_type: 1
    GRIB_level_type_name: surface
    GRIB_interval_stat_type: Average
    GRIB_VectorComponentFlag: easterlyNortherlyRelative
unlimited dimensions: 
current shape = (5113, 11, 5, 9, 16)
filling on

In [6]:
dswrf.shape

(5113, 11, 5, 9, 16)

In [7]:
dswrf.dimensions

(u'time', u'ens', u'fhour', u'lat', u'lon')

In [8]:
for d in data.dimensions.values():
    print d

<type 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 5113

<type 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 9

<type 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 16

<type 'netCDF4._netCDF4.Dimension'>: name = 'ens', size = 11

<type 'netCDF4._netCDF4.Dimension'>: name = 'fhour', size = 5



In [9]:
lat = data.variables['lat']
lat.shape

(9,)

In [10]:
l = lat[:]
l

array([ 31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.], dtype=float32)

In [11]:
np.where(l == 33)

(array([2]),)

In [12]:
np.where(l == 33.)[0][0]

2

In [13]:
lon = data.variables['lon']
n = lon[:]
n

array([ 254.,  255.,  256.,  257.,  258.,  259.,  260.,  261.,  262.,
        263.,  264.,  265.,  266.,  267.,  268.,  269.], dtype=float32)

In [14]:
np.where(n == 260.)[0][0]

6

So, to find the data for a point at (33,260)

In [15]:
p_33_260 = dswrf[:,:,:,np.where(l == 33)[0][0],np.where(n == 260)[0][0]]

In [16]:
p_33_260.shape

(5113, 11, 5)

In [17]:
p = np.mean(p_33_260, axis = 1)

In [18]:
p.shape

(5113, 5)

In [19]:
p[1000,2]

296.81818

In [20]:
p = p_33_260.reshape(p_33_260.shape[0], p_33_260.shape[1] * p_33_260.shape[2])

In [21]:
p.shape

(5113, 55)

In [22]:
np.savetxt('33_260.csv',p,delimiter=',',fmt="%.06f")

In [23]:
dswrf[0,0,0,0,0]

0.0

In [24]:
dswrf1D = np.reshape(dswrf,-1)

In [25]:
len(dswrf1D)

40494960

In [26]:
dswrf1D.shape

(40494960,)

In [27]:
values = data.variables.values()

In [28]:
models = values[4][:]

In [29]:
models = [str(val) for val in models]

In [30]:
times = values[5][:]
times

array([12, 15, 18, 21, 24], dtype=int32)

In [31]:
import datetime
datetime.datetime.strptime(str(values[1][0]),"%Y%m%d%H")

datetime.datetime(1994, 1, 1, 0, 0)

In [32]:
import datetime
values = data.variables.values()
models = [str(val) for val in values[4][:]]
times = [datetime.time(val%24) for val in values[5][:]]
dates = [datetime.datetime.strptime(str(val), "%Y%m%d%H") for val in values[1][:]]

In [33]:
import pandas as pd

In [34]:
import itertools
new_indices = list(itertools.product(dates, models, times, values[2][:],values[3][:]))

In [35]:
len(new_indices)

40494960

In [ ]:
dswrf_df = pd.DataFrame(dswrf1D)

In [ ]:
import numpy as np

In [ ]:
index_frame = pd.DataFrame(new_indices)

In [ ]:
index_frame.head()

In [ ]:
dswrf_df.columns=['total_precip']

In [ ]:
dswrf_df.head()

In [ ]:
index_frame.columns=['date','model','time','latitude','longitude']

In [ ]:
index_frame.head()

In [ ]:
all_info = pd.merge(index_frame,precip_df,left_index=True, right_index=True)

In [ ]:
type(new_indices)

In [ ]:
one_loc_one_day = all_info[(all_info['latitude'] == 31) & (all_info['longitude'] == 254) & (all_info['date'] == '2004-03-07')]

In [ ]:
one_loc_one_day

In [36]:
dswrf1D

array([   0.,    0.,    0., ...,  120.,  120.,  150.], dtype=float32)

In [ ]:
type(np.asarray(new_indices))

In [37]:
arrayed_index = np.asarray(new_indices)

In [44]:
stacked = np.hstack((arrayed_index, dswrf1D[:,None]))